Basic demonstration of the quantum chemistry code I have written. It seems to be fairly quick, but uses quite a bit of ram. One trick which I certainly did not exploit is that some parts of the environments are simply the harmitian conjugate of other parts, and the environments can therefore be compressed further (and should be twice as cheap to calculate).

In [1]:
using LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();


ERROR: LoadError: 

UndefVarError: `EuclideanSpace` not defined
Stacktrace:
 [1] top-level scope
   @ ~/.julia/packages/MPSKit/BToBw/src/states/abstractmps.jl:5
 [2] include(mod::Module, _path::String)
   @ Base ./Base.jl:457
 [3] include(x::String)
   @ MPSKit ~/.julia/packages/MPSKit/BToBw/src/MPSKit.jl:1
 [4] top-level scope
   @ ~/.julia/packages/MPSKit/BToBw/src/MPSKit.jl:73
 [5] include
   @ ./Base.jl:457 [inlined]
 [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base ./loading.jl:2010
 [7] top-level scope
   @ stdin:2
in expression starting at /home/maavdamm/.julia/packages/MPSKit/BToBw/src/states/abstractmps.jl:5
in expression starting at /home/maavdamm/.julia/packages/MPSKit/BToBw/src/MPSKit.jl:1
in expression starting at stdin:2


ErrorException: Failed to precompile MPSKit [bb1c41ca-d63c-52ed-829e-0820dda26502] to "/home/maavdamm/.julia/compiled/v1.9/MPSKit/jl_Xaz5eo".

FCIDUMP is a well known (though badly documented) format to store the electronic integrals. I have written an experimental parser to load in such files:

In [2]:
(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.STO3G.FCIDUMP")

UndefVarError: UndefVarError: `parse_fcidump` not defined

ERI is the Electronic Repulsion Integral, K is the two-body term, E0 is an energy shift, NORB is the number of orbitals (mps length), NELEC is the number of electrons and MS2 is the total spin.

fcidump files can contain orbital symmetries, but I ignore these.

Using these variables, we can instantiate the hamiltonian. Ideally we'd use real numbers, but tensorkit uses a complex default element type for the braidingtensor, and this causes issues.

In [3]:
(ham,_) = fused_quantum_chemistry_hamiltonian(E0,K,ERI,ComplexF64);

UndefVarError: UndefVarError: `fused_quantum_chemistry_hamiltonian` not defined

The second return value contains some metadata which can be ignored. Instantiate the state:

In [4]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-NELEC,MS2//2,mod1(-NELEC,2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -NELEC:0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,ComplexF64,NORB,psp,virtual;left,right);

UndefVarError: UndefVarError: `Vect` not defined

We also need to instantiate the environments, because MPSKit will otherwise call the default environment constructor which keeps everything in ram (I think I even disabled that constructor). To create an environment that lives on disk, we can use:

In [ ]:
envs = disk_environments(st,op);

Let's call DMRG!

In [ ]:
@time (st,envs) = find_groundstate(st,op, DMRG2(trscheme=truncdim(100), verbose = true),envs);